In [7]:
from datetime import datetime
import sys
import time
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
from PIL import Image
import html
from markdownify import markdownify 
import folium




endpoint_url = "https://computational-publishing-service.wikibase.cloud/query/sparql"

endpoint_url = "https://query.wikidata.org/sparql"

query = """
# Get images of brutalist buildings
SELECT DISTINCT ?item ?itemLabel ?image ?placeLabel ?countryLabel (YEAR(?inception) AS ?year) ?latitude ?longitude ?itemDescription WHERE {
  ?item wdt:P31 wd:Q41176;
    wdt:P149 wd:Q994776;
    wdt:P625 ?coordinates;
    wdt:P571 ?inception;
    wdt:P131 ?place;
    wdt:P17 ?country;
    wdt:P18 ?image.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,[AUTO_LANGUAGE]". }
  BIND(geof:latitude(?coordinates) AS ?latitude).
  BIND(geof:longitude(?coordinates) AS ?longitude).
}
"""


locations = []

def run_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST") #this NEEDS to be added to get results (not included in the wikibase python example code)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def get_img(partOfItem_id):

    print('<script src="https://unpkg.com/masonry-layout@4/dist/masonry.pkgd.min.js"></script>')
    print('<script src="https://unpkg.com/imagesloaded@4/imagesloaded.pkgd.min.js"></script>')
    print("<link rel='stylesheet' href='./masonrydesign.css'>")
    print('<div class="img_grid">')
    q = query

    results = run_query(endpoint_url, q)

    for item in results["results"]["bindings"]:    

      alt='image of a ' + item['itemDescription']['value'] if 'itemDescription' in item else 'image of a brutalist building'
      title=item['itemLabel']['value'] if 'itemLabel' in item else ''
      year=item['year']['value'] if 'year' in item else ''
      latitude=item['latitude']['value'] if 'latitude' in item else ''
      longitude=item['longitude']['value'] if 'longitude' in item else ''
      place=item['placeLabel']['value']
      country=item['countryLabel']['value'] 
      item_url=item['item']['value'] if 'item' in item else ""
      image_url=item['image']['value'] if 'image' in item else ''

      locations.append((title, latitude, longitude))

#       openpop = f'openPopover("{image_url}")'
#       closepop = f"closePopover('{image_url}')"
#       openinfo = f'openInfo("{item_url}")'
#       closeinfo = f"closeInfo('{item_url}')"


#       print(f"<div class='info-wd btn-group' id=\"{item_url}\" style='display:none'><a href='{item_url}'><button class='btn btn-light'>{title}</button></a><button type='button' class='btn btn-secondary'>{place}, {country}</button><button type='button' class='btn btn-secondary'>Built: {year}</button></div>")
#       print(f"<img title='{title}' class='img_small' src='{image_url}' popovertarget='{image_url}' onclick='{openpop};{openinfo}' alt='{alt}'>")
#       #print(f'<input type="image" class="img_small" src="{image_url}" popovertarget="{image_url}" onclick="{onclickopen}" />')
#       print(f'<img popover class="img_popover" src="{image_url}" id="{image_url}" onclick="{closepop},{closeinfo}" alt="{alt}" style="display: none;">')
#     print('</div>')


#     print("""
# <script>
#   var grid = document.querySelector('.img_grid');
          
#   grid.style.display = 'none';

#   // Initialize Masonry, but delay layout until images are loaded
#   var msnry = new Masonry(grid, {
#     itemSelector: '.img_small',
#     percentPosition: true,
#   });
  

#   // Use imagesLoaded to fix layout issues as images load
#   imagesLoaded(grid, function() {
#     grid.style.display = 'block';
#     msnry.layout();
#   });

#   function openPopover(imageUrl) {
#     var popover = document.getElementById(imageUrl);
#     var body = document.querySelector('body');
#     // Show the popover
#     popover.style.display = 'block';
#     body.classList.add("stop-scrolling");
#   };
          
#   function openInfo(itemUrl) {
#     var popover = document.getElementById(itemUrl);
#     var body = document.querySelector('body');
#     // Show the popover
#     popover.style.display = 'flex';
#   };

#   function closePopover(imageUrl) {
#     var popover = document.getElementById(imageUrl);
#     var body = document.querySelector('body');
#     popover.style.display = 'none';
#     body.classList.remove("stop-scrolling");
#   };
          
#   function closeInfo(itemUrl) {
#     var popover = document.getElementById(itemUrl);
#     var body = document.querySelector('body');
#     popover.style.display = 'none';
#   };
          
# </script>
#           """)
    
#| label: mappy

# Add some points (replace these with your coordinates)

# for lat, lon in locations:
#     folium.Marker([lat, lon]).add_to(m)

# Show the map





# def get_map():
#   print('''<br><br><iframe 
#   style="width: 80vw; height: 80vh; border: none;" 
#   src="https://query.wikidata.org/sparql?query=%23defaultView%3AMap%7B%22hide%22%3A%5B%22%3Fcoordinates%22%2C%20%22%3FplaceLabel%22%2C%20%22%3FcountryLabel%22%5D%7D%0ASELECT%20DISTINCT%20%3Fitem%20%3FitemLabel%20%3Fimage%20%3Fcoordinates%20%3FplaceLabel%20%3FcountryLabel%20(YEAR(%3Finception)%20AS%20%3Fyear)%20%3FitemDescription%20WHERE%20%7B%0A%20%20%3Fitem%20wdt%3AP31%20wd%3AQ41176%3B%0A%20%20%20%20wdt%3AP149%20wd%3AQ994776%3B%0A%20%20%20%20wdt%3AP625%20%3Fcoordinates%3B%0A%20%20%20%20wdt%3AP571%20%3Finception%3B%0A%20%20%20%20wdt%3AP131%20%3Fplace%3B%0A%20%20%20%20wdt%3AP17%20%3Fcountry%3B%0A%20%20%20%20wdt%3AP18%20%3Fimage.%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22en%22.%20%7D%0A%7D" 
#   sandbox="allow-scripts allow-same-origin allow-popups">
#   </iframe>''')


In [8]:
get_img("")

m = folium.Map(location=[40.7128, -74.0060], zoom_start=3)


for name, lat, lon in locations:
    folium.Marker(
        location=[lat, lon],
        popup=f"<b>{name}</b>",  # Clickable label
        tooltip=name  # Hover label
    ).add_to(m)
    

m
#get_map()
#get_text()

<script src="https://unpkg.com/masonry-layout@4/dist/masonry.pkgd.min.js"></script>
<script src="https://unpkg.com/imagesloaded@4/imagesloaded.pkgd.min.js"></script>
<link rel='stylesheet' href='./masonrydesign.css'>
<div class="img_grid">


In [9]:
#get_graph()